---
title: "Accelerate, three powerful sublibraries for PyTorch"
author: "Zachary Mueller"
format: revealjs
---

## Getting up

- Turn off alarm
- Get out of bed

## Going to sleep

In [1]:
print("Hello world!")

Hello world!
